In [3]:
import plotly.graph_objects as go
import pandas as pd
import matplotlib.pyplot as plt
import findspark
import os
from datetime import datetime
from functools import reduce
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
import numpy as np
findspark.init()

In [4]:
os.environ["SPARK_HOME"]=r"D:\spark\spark"
os.environ["HADOOP_HOME"]=r"D:\spark\spark\hadoop\bin"
os.environ["JAVA_HOME"]=r"D:\spark\jdk-21"

In [5]:
jarsql = r"C:\leads\venv\Lib\site-packages\pyspark\mssql-jdbc-9.4.0.jre11.jar"

In [6]:

from pyspark.sql import SparkSession
from os.path import abspath
from pyspark import SparkContext, SparkConf
from pyspark.sql.functions import col, regexp_replace, when, from_unixtime, lit, concat,to_date,unix_timestamp
import pyspark.sql.functions as F

jarsql = r"C:\leads\venv\Lib\site-packages\pyspark\mssql-jdbc-9.4.0.jre11.jar"
warehouse_location = abspath('spark-warehouse')

conf = SparkConf()
conf.set("spark.master","local[*]")
conf.set("spark.executor.memory", "4g")
conf.set("spark.driver.memory", "4g")
conf.set('spark.driver.extraClassPath', jarsql)
conf.set('spark.executor.extraClassPath', jarsql)
spark = SparkSession.builder\
    .config(conf=conf)\
    .config("spark.sql.warehouse.dir", warehouse_location)\
    .appName("Testing PySpark Example")\
    .getOrCreate()


sc = spark.sparkContext

In [7]:
file_path = r"D:\analise_produtos\excelfiles\super_loja.csv"
df = spark.read.option("sep", ";").option("header", "true").option("inferSchema", "true").csv(file_path)

In [8]:
df.show()

+----------+--------------+--------------+-------------+-------------+-------------+-----------------+--------------------+----------------+----------------+-----------+-------+---------------+--------------------+-------------+--------------------+------+----------+--------+------+
|ID da fila|  ID do pedido|Data do pedido|Data de envio|Modo de envio|ID do cliente|  Nome do cliente|            Segmento|          Cidade|          Estado|Pa�s/Regi�o| Regi�o|  ID do produto|           Categoria|Sub-categoria|     Nome do produto|Vendas|Quantidade|Desconto| Lucro|
+----------+--------------+--------------+-------------+-------------+-------------+-----------------+--------------------+----------------+----------------+-----------+-------+---------------+--------------------+-------------+--------------------+------+----------+--------+------+
|         1|MX-2021-143658|    02/10/2021|   06/10/2021|Classe padr�o|     SR-20575|      Sofia Rocha|              Varejo|Cidade do M�xico|Distrito

In [9]:
lista_columns_lower = [i for i in df.columns]
for lista in lista_columns_lower:
    
    df = df.withColumnRenamed(
        lista, lista.replace(" ","_").replace(
            "/","_").replace("�o","ao").replace("a�","ai").lower().strip())
df.show()
    

+----------+--------------+--------------+-------------+-------------+-------------+-----------------+--------------------+----------------+----------------+-----------+-------+---------------+--------------------+-------------+--------------------+------+----------+--------+------+
|id_da_fila|  id_do_pedido|data_do_pedido|data_de_envio|modo_de_envio|id_do_cliente|  nome_do_cliente|            segmento|          cidade|          estado|pais_regiao| regiao|  id_do_produto|           categoria|sub-categoria|     nome_do_produto|vendas|quantidade|desconto| lucro|
+----------+--------------+--------------+-------------+-------------+-------------+-----------------+--------------------+----------------+----------------+-----------+-------+---------------+--------------------+-------------+--------------------+------+----------+--------+------+
|         1|MX-2021-143658|    02/10/2021|   06/10/2021|Classe padr�o|     SR-20575|      Sofia Rocha|              Varejo|Cidade do M�xico|Distrito

In [10]:
lista_floats = ["vendas","desconto","lucro"]
for col_name in lista_floats:
    df = df.withColumn(col_name, regexp_replace(col(col_name), ",", ".").cast("float"))

In [11]:
date_columns = ["data_do_pedido", "data_de_envio"]
for col_name in date_columns:
    df = df.withColumn(col_name, when(col(col_name) != "NULL", col(col_name)))


In [12]:
date_format = "dd/MM/yyyy"
for col_name in date_columns:
    df = df.withColumn(col_name, to_date(col(col_name), date_format))

In [13]:
tab_vendas = df.select("data_do_pedido","vendas","quantidade","desconto","lucro",
          "data_de_envio","segmento","regiao","categoria","sub-categoria","id_do_produto","nome_do_produto","id_do_pedido","id_do_cliente","estado","cidade")

In [14]:
df = df.withColumn("quantidade", df["quantidade"].cast("int"))
df.show()

+----------+--------------+--------------+-------------+-------------+-------------+-----------------+--------------------+----------------+----------------+-----------+-------+---------------+--------------------+-------------+--------------------+------+----------+--------+-------+
|id_da_fila|  id_do_pedido|data_do_pedido|data_de_envio|modo_de_envio|id_do_cliente|  nome_do_cliente|            segmento|          cidade|          estado|pais_regiao| regiao|  id_do_produto|           categoria|sub-categoria|     nome_do_produto|vendas|quantidade|desconto|  lucro|
+----------+--------------+--------------+-------------+-------------+-------------+-----------------+--------------------+----------------+----------------+-----------+-------+---------------+--------------------+-------------+--------------------+------+----------+--------+-------+
|         1|MX-2021-143658|    2021-10-02|   2021-10-06|Classe padr�o|     SR-20575|      Sofia Rocha|              Varejo|Cidade do M�xico|Distr

In [15]:
tab_sales = tab_vendas.select("data_do_pedido","vendas")

In [16]:
tab_sales.show()

+--------------+------+
|data_do_pedido|vendas|
+--------------+------+
|    2021-10-02| 130.8|
|    2019-10-16|2521.6|
|    2019-10-16|1932.8|
|    2019-10-16| 354.4|
|    2019-10-16| 716.0|
|    2019-10-16| 561.2|
|    2020-09-28| 561.2|
|    2020-09-28|3446.4|
|    2020-09-28| 973.6|
|    2020-09-28|3415.2|
|    2020-09-28| 120.6|
|    2020-03-06| 207.6|
|    2020-03-06|2106.4|
|    2021-09-24| 801.0|
|    2021-10-18|1326.4|
|    2021-10-18| 129.4|
|    2020-06-27| 188.4|
|    2020-06-27|3082.8|
|    2020-06-27|401.76|
|    2020-06-27| 87.84|
+--------------+------+
only showing top 20 rows



In [17]:
start_date = datetime(2021, 1, 1)
filtered_df = tab_vendas.filter(col("data_do_pedido") >= start_date)

In [18]:
vendas_df = filtered_df.toPandas()
vendas_df.dropna(inplace=True)

In [19]:
vendas_df.columns

Index(['data_do_pedido', 'vendas', 'quantidade', 'desconto', 'lucro',
       'data_de_envio', 'segmento', 'regiao', 'categoria', 'sub-categoria',
       'id_do_produto', 'nome_do_produto', 'id_do_pedido', 'id_do_cliente',
       'estado', 'cidade'],
      dtype='object')

In [20]:
vendas = vendas_df.groupby(['data_do_pedido'])['vendas'].sum().reset_index()
vendas

,data_do_pedido,vendas
0,2021-01-01,14151.386719
1,2021-01-03,5778.340332
2,2021-01-04,1297.599976
3,2021-01-06,1507.199951
4,2021-01-07,3857.199951
...,...,...
312,2021-12-27,33777.378906
313,2021-12-28,2719.680176
314,2021-12-29,31637.273438
315,2021-12-30,28163.041016


In [21]:
vendas['aumento'] = vendas['vendas'].diff()

In [22]:
vendas['aceleracao'] = vendas['aumento'].diff()

In [23]:
vendas

,data_do_pedido,vendas,aumento,aceleracao
0,2021-01-01,14151.386719,NaN,NaN
1,2021-01-03,5778.340332,-8373.046875,NaN
2,2021-01-04,1297.599976,-4480.740234,3892.306641
3,2021-01-06,1507.199951,209.599976,4690.340332
4,2021-01-07,3857.199951,2350.000000,2140.399902
...,...,...,...,...
312,2021-12-27,33777.378906,-32708.816406,-60274.031250
313,2021-12-28,2719.680176,-31057.699219,1651.117188
314,2021-12-29,31637.273438,28917.593750,59975.292969
315,2021-12-30,28163.041016,-3474.232422,-32391.826172


In [30]:


def plot_comparacao(x, y1, y2, y3, dataset, titulo):
    fig1 = go.Figure()
    fig2 = go.Figure()
    fig3 = go.Figure()

    fig1.add_trace(go.Scatter(x=dataset[x], y=dataset[y1], mode='lines', name=y1))
    fig1.update_layout(
        title=titulo,
        xaxis_title=x,
        yaxis_title="Valores",
        xaxis=dict(showline=True, showgrid=False),
        yaxis=dict(showline=True, showgrid=False),
        template="plotly_dark", 
        legend=dict(font=dict(family="Arial", size=12, color="white")),
        hovermode="x",  
    )

   
    fig2.add_trace(go.Scatter(x=dataset[x], y=dataset[y2], mode='lines', name=y2))
    fig2.update_layout(
        title=titulo,
        xaxis_title=x,
        yaxis_title="Valores",
        xaxis=dict(showline=True, showgrid=False),
        yaxis=dict(showline=True, showgrid=False),
        template="plotly_dark",
        legend=dict(font=dict(family="Arial", size=12, color="white")),
        hovermode="x",
    )

    
    fig3.add_trace(go.Scatter(x=dataset[x], y=dataset[y3], mode='lines', name=y3))
    fig3.update_layout(
        title=titulo,
        xaxis_title=x,
        yaxis_title="Valores",
        xaxis=dict(showline=True, showgrid=False),
        yaxis=dict(showline=True, showgrid=False),
        template="plotly_dark",
        legend=dict(font=dict(family="Arial", size=12, color="white")),
        hovermode="x",
    )

    fig1.show()
    fig2.show()
    fig3.show()


plot_comparacao('data_do_pedido', 'vendas', 'aumento', 'aceleracao',
                vendas, 'Análise das vendas das Lojas 2021')

In [25]:
scaler = MinMaxScaler()
vendas['vendas'] = scaler.fit_transform(vendas['vendas'].values.reshape(-1, 1))

In [27]:
def create_sequences(data, seq_length):
    sequences, labels = [], []
    for i in range(len(data) - seq_length):
        seq = data[i:i+seq_length]
        label = data[i+seq_length]
        sequences.append(seq)
        labels.append(label)
    return np.array(sequences), np.array(labels)


seq_length = 10 
epochs = 50


train_sequences, train_labels = create_sequences(vendas['vendas'].values, seq_length)

In [ ]:
"""
LSTM 
"""

In [29]:
model = Sequential()
model.add(LSTM(64, activation='relu', return_sequences=True, input_shape=(seq_length, 1))) 
model.add(LSTM(32, activation='relu', return_sequences=True))
model.add(LSTM(16, activation='relu')) 
model.add(Dense(1)) 

model.compile(optimizer='adam', loss='mse')
model.fit(train_sequences, train_labels, epochs=epochs, batch_size=64)

Epoch 1/50
5/5 [==============================] - 3s 11ms/step - loss: 0.0865
Epoch 2/50
5/5 [==============================] - 0s 11ms/step - loss: 0.0745
Epoch 3/50
5/5 [==============================] - 0s 10ms/step - loss: 0.0641
Epoch 4/50
5/5 [==============================] - 0s 10ms/step - loss: 0.0527
Epoch 5/50
5/5 [==============================] - 0s 13ms/step - loss: 0.0419
Epoch 6/50
5/5 [==============================] - 0s 11ms/step - loss: 0.0354
Epoch 7/50
5/5 [==============================] - 0s 11ms/step - loss: 0.0367
Epoch 8/50
5/5 [==============================] - 0s 10ms/step - loss: 0.0354
Epoch 9/50
5/5 [==============================] - 0s 10ms/step - loss: 0.0345
Epoch 10/50
5/5 [==============================] - 0s 10ms/step - loss: 0.0350
Epoch 11/50
5/5 [==============================] - 0s 10ms/step - loss: 0.0347
Epoch 12/50
5/5 [==============================] - 0s 12ms/step - loss: 0.0341
Epoch 13/50
5/5 [==============================] - 0s 10ms/st

In [ ]:
"""previsões (4 anos à frente)"""
n_previsoes = 4 * 365 
ultimas_sequencias = train_sequences[-1].reshape(1, seq_length, 1)
previsoes = []
datas_previsao = []

for _ in range(n_previsoes):
    previsao = model.predict(ultimas_sequencias)
    previsoes.append(scaler.inverse_transform(previsao)[0][0])

    
    ultimas_sequencias = np.roll(ultimas_sequencias, shift=-1, axis=1)
    ultimas_sequencias[0, -1, 0] = previsao

    """Gere a data para a previsão (assumindo que é diária)"""
    data_atual = pd.to_datetime(vendas.index[-1])
    data_previsao = data_atual + pd.DateOffset(days=1)
    datas_previsao.append(data_previsao)


df_previsoes = pd.DataFrame({'Data': datas_previsao, 'Valor Real': previsoes[:-1], 'Previsão': previsoes[1:]})


print(df_previsoes)

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(vendas['data'], vendas['vendas'], label='Vendas reais', linestyle='-', marker='o')
plt.plot(df_previsoes['Data'], df_previsoes['Valor Real'], label='Valor Real', linestyle='--')
plt.plot(df_previsoes['Data'], df_previsoes['Previsão'], label='Previsão', linestyle='--')
plt.xlabel('Data')
plt.ylabel('Vendas')
plt.legend()
plt.show()
